# Content table <a name="content"></a>
### 1. [Read images, videos, webcam](#read_image)
### 2. [Basic function](#basic)
### 3. [Resize and crap](#resize)
### 4. [Shapes and Texts](#shape)
### 5. [Wrap perspective](#wrap)
### 6. [Join images](#join)
### 7. [Color dectection](#color)
### 8. [Contour / Shape dectection](#contour)
### 9. [Object detection](#object)

--------------------------------------------------

## Read images, videos, webcam <a name="read_image"></a>

[Back to content table](#content)

* `cv2.imread(source)`  
    source: The path of the image

  
* `cv2.imshow(name, img)`  
    * name: Name of the window  
    * img: Image to show

In [ ]:
import cv2

img = cv2.imread("resources/scene.jpg")


cv2.imshow("Scene", img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import numpy as np
import cv2

img = cv2.imread("resources/food.jpg")

# Add noise to the image to see how blurring works
gaussian = np.random.normal(0, 0.3, img.shape) * 255
img_noisy = img + gaussian
img_noisy = np.clip(img_noisy, 0, 255).astype(np.uint8)

# Convert the image to gray scale
img_noisy_gray = cv2.cvtColor(img_noisy, cv2.COLOR_BGR2GRAY)

'''
Blur the image
'''
# Average blur
img_average = cv2.blur(img_noisy_gray, (3, 3))

# Median blur
img_median = cv2.medianBlur(img_noisy_gray, 3)

# Gaussian blur
img_gaussian = cv2.GaussianBlur(img_noisy_gray, (3, 3), 0)

# Stack the image (Will introduce latter, the user-defined function)
result = stackImages(0.5, [[img, img_noisy, img_noisy_gray], [img_average, img_median, img_gaussian]])

cv2.imshow("result", result)


cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

cv2.imwrite("result/blur.jpg", result)

* `cv2.ViedoCapture(source)` <- cap  
    1. source: The path for video  
      If source is 0, then use the default webcame
  
    
* `cap.read()`  
   1. Return (state, image)
       * state: Check whether the camera is working  
       * image: Image read from camera
       
         
* `cap.release()`  
    Close the camera

In [ ]:
import cv2

cap = cv2.VideoCapture("resources/test.mp4")

'''
Video is the sequence of images.
We need while loop to show the video
'''
while True:
    success, img = cap.read()
    cv2.imshow("Video", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break;
        
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import cv2

# Use the default webcame
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(10, 100)

'''
Video is the sequence of images.
We need while loop to show the video
'''
while True:
    success, img = cap.read()
    
    # To ensure the camera can successfully read the image
    if (not success):
        continue
        
    cv2.imshow("Video", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break;

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!

## Basic function <a name="basic"></a>

[Back to content table](#content)


* `cv2.cvtColor(img, code)`  
    * img: Image going to convert
    * code: Type for the conversion. See the [documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html#cvtcolor)
      
    
* `cv2.GaussianBlur(img, kernel_size, sigma_X)`  
    * img: Image to be propcessed
    * kernel_size: Size for the filter matrix
    * sigma: Standard deviation of the direction of X  
    [Here](https://datacarpentry.org/image-processing/06-blurring/) is how GaussianBlur works!
    
      
* `cv2.canny(img, threshold1, threshold2)`  
    * img: Image to be processed  
    * threshold1: First threshold  
    * threshold2: Second threshold.  
    [Here](https://towardsdatascience.com/canny-edge-detection-step-by-step-in-python-computer-vision-b49c3a2d8123) is how Canny works!
      
   
   [Here](https://www.mathworks.com/help/images/morphological-dilation-and-erosion.html) is how both dilation and erosion work!
* `cv2.dilate(img, kernel, iteration)`  
    The value of the output pixel is the **maximum** value of all pixels in the neighborhood.  
    * img: Image to be processed  
    * kernel: The size of checking kernel  
    * iteration: Times for doing the filter  
    
      
* `cv2.eroded(img, kernel, iteration)`  
    The value of the output pixel is the **minimum** value of all pixels in the neighborhood.  
    * img: Image to be processed  
    * kernel: The size of checking kernel  
    * iteration: Times for doing the filter  

In [ ]:
import cv2
import numpy as np

img = cv2.imread("resources/car.jpg")

# Change the image to gray scale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Blur the image to reduce noise
img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)

'''
# Edge detection
'''
# Canny detector
img_canny = cv2.Canny(img_blur, 100, 100)

# LoG detector (Use img, not img_blur)
img_log = cv2.Laplacian(img_gray, -1, cv2.CV_16S)

# Stack the image
result = stackImages(0.5, [img, img_canny, img_log])

cv2.imshow("Result", result)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

cv2.imwrite("Edge_total.jpg", result)

In [ ]:
import cv2
import numpy as np

img = cv2.imread("resources/shapes.jpg")

# Change the image to gray scale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Blur the image
img_blur = cv2.GaussianBlur(img_gray, (7, 7), 0)

# Edge detection
img_thresh = cv2.threshold(img_blur, 200, 1, cv2.THRESH_BINARY_INV)[1].astype(np.float)

# Define the kernel
kernel = np.ones((69, 69), np.uint8)

# First erase the noise
img_eroded = cv2.erode(img_thresh, kernel, iterations=1)

# Back to the original feature
img_dilation = cv2.dilate(img_eroded, kernel, iterations=1)

# Show the above image
result = np.hstack([img_thresh, img_eroded, img_dilation])

cv2.imshow("Result", result)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import cv2
import numpy as np

img = cv2.imread("resources/beauty.jpg")

# Change the image to gray scale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Blur the image
img_blur = cv2.GaussianBlur(img_gray, (7, 7), 0)

# Edge detection
img_canny = cv2.Canny(img, 100, 100)

# Define the kernel
kernel = np.ones((5, 5), np.uint8)

# Need gray scale
img_dilation = cv2.dilate(img_canny, kernel, iterations=1)

# Need gray scale
img_eroded = cv2.erode(img_dilation, kernel, iterations=1)


# Show the above image
cv2.imshow("Gray image", img_gray)
cv2.imshow("Blur image", img_blur)
cv2.imshow("Canny image", img_canny)
cv2.imshow("Dilation image", img_dilation)
cv2.imshow("Erosion image", img_eroded)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!

## Resize and crap <a name="resize"></a>

[Back to content table](#content)

* `cv2.resize(img, (width, height))`  
    * img: Image to be processed
    * (width, height): The size of the new image

In [ ]:
import cv2

img = cv2.imread("resources/food.jpg")

# Print out the size of the image
print(img.shape)

'''
# Resize the image
size: (width, heigth)
width: 500
height: 200
'''
img_resize = cv2.resize(img, (500, 200))

# Print out the new size of the image
print(img_resize.shape)


'''
# Crop the image
target region: (height, width)
height pixel: 300_700
width pixel: 400_100
'''
img_cropped = img[300:900, 400:1000]

# Show the image
cv2.imshow("Image", img)
cv2.imshow("Resize image", img_resize)
cv2.imshow("Cropped image", img_cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import cv2

img = cv2.imread("resources/food.jpg")

'''
# Crop the image
target region: (height, width)
height pixel: 500~900
width pixel: 300~1000
'''
img_cropped = img[500:900, 300:1000]

# Show the image
cv2.imshow("Image", img)
cv2.imshow("Cropped image", img_cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

cv2.imwrite("result/crop.jpg", img_cropped)

In [ ]:
# Do what you want to test!

## Shapes and Texts <a name="shape"></a>

[Back to content table](#content)

* `cv2.line(img, start_point, end_point, color, thickness)`  
    * img: Image to add line  
    * start_point: Pixel to start line  
    * end_point: Pixel to end line  
    **Both start_point and end_point should be used in (width_pixel, heigth_pixel).** 
    
    * color: Color of line  
    * thickness: The thickness of line
      
    
* `cv2.rectangle(img, upper_left, lower_right, color, thickness))`  
    * img: Image to add rectangle. 
    * upper_left: Pixel to start rectangle  
    * lower_right: Pixel to end rectangle  
    **Both upper_left and lower_right should be used in (width_pixel, heigth_pixel).** 
    
    * color: Color of rectangle  
    * thickness: Thickness of rectangle. If use cv2.FILLED, all the area will be colored.
      
    
* `cv2.circle(img, orgin, redius, color, thickness)`  
    * origin: The origin place of the circle  
    * redius: Redius of the circle  
    * thickness: Thickness of the circle  
      
    
* `cv2.putText(img, text, start_place, font type, scale, color, thickness)`  
    * text: Text to be put on  
    * start_place: buttom-left corner of the text  
    * font type: The type of the text  
    * scale: The scale of the text  

In [ ]:
import cv2
import numpy as np

'''
# Create a black image
0: black
255: white
'''
img_black = np.zeros((512, 512), np.uint8)

# Create color image, need 3 channels
img_blue = np.zeros((512, 512, 3), np.uint8)
img_blue[:] = [255, 0, 0]

'''
Try out
img_blue[200:300, 300, 400] = [255, 0, 0]
'''

# Add green line
img_line = np.zeros((512, 512, 3), np.uint8)
cv2.line(img_line, (0, 0), (img.shape[1], img.shape[0]), (0, 255, 0), 3)

## Add rectangle
img_rec = np.zeros((512, 512, 3), np.uint8)
cv2.rectangle(img_rec, (0, 0), (250, 350), (0, 0, 255), 3)
cv2.rectangle(img_rec, (250, 350), (img.shape[1], img.shape[0]), (255, 0, 0), cv2.FILLED)

## Add Circle
img_cir = np.zeros((512, 512, 3), np.uint8)
cv2.circle(img_cir, (250, 250), 100, (255, 0, 255), cv2.FILLED)

## Add Text
img_text = np.zeros((512, 512, 3), np.uint8)
cv2.putText(img_text, "OpenCV", (300, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 3)

# Show the result
cv2.imshow("Black image", img_black)
cv2.imshow("Blue image", img_blue)
cv2.imshow("Line", img_line)
cv2.imshow("Rectangle", img_rec)
cv2.imshow("Circle", img_cir)
cv2.imshow("Text", img_text)


cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!

## Wrap perspective  <a name="wrap"></a>

[Back to content table](#content)


* `cv2.getPerspectiveTransform(pst1, pst2)`  
    * pst1: np.float([[old1], [old2], [old3], [old4]])  
    * pst2: np.float([[new1], [new2], [new3], [new4]]) 
     
       
* `cv2.warpPerspective(img, matrix, (width, height))`  
    * img: Old image  
    * matrix: Result came from cv2.getPerspective()  
    * (width, height): new image size  


In [ ]:
import cv2
import numpy as np

img = cv2.imread("resources/pocker.jpg")

cv2.imshow("Image", img)

# New size of the output
width, height = 250, 350

'''
pst1: old position
pst2: new position
transform pst1 -> pst2
'''
pts1 = np.float32([[239, 103], [356, 146], [179, 277], [296, 320]])
pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

pts1 = np.float32([[356, 146], [239, 103], [179, 277], [296, 320]])

matrix = cv2.getPerspectiveTransform(pts1, pts2)
img_output = cv2.warpPerspective(img, matrix, (width, height))

cv2.imshow("New image", img_output)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!

## Join images <a name="join"/></a>

[Back to content table](#content)

In [ ]:
import cv2
import numpy as np

img = cv2.imread("resources/food.jpg")

# horizotal stack
img_hor = np.hstack((img, img))

# Vertival stack
img_ver = np.vstack((img, img))

# Show the result
cv2.imshow("Horizontal image", img_hor)
cv2.imshow("Vertical image", img_ver)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])

    rowsAvailable = isinstance(imgArray[0], list)
    
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    
    if rowsAvailable:
        for x in range (0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                    
                if len(imgArray[x][y].shape) == 2: 
                    imgArray[x][y]= cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
                    
        imageBlank = np.zeros((height, width, 3), np.uint8)
        
        hor = [imageBlank] * rows
        
        # First put each horizontal image to its horizontal place
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        
        # Stack each array to the vertical format
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
                
            if len(imgArray[x].shape) == 2: 
                imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
                
        hor = np.hstack(imgArray)
        ver = hor
        
    return ver

In [ ]:
import cv2
import numpy as np
 
img = cv2.imread('resources/food.jpg')
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Stack image
img_stack = stackImages(0.5, [[img, img_gray, img],[img, img, img]])

# Show the image
cv2.imshow("ImageStack",img_stack)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!

## Color dectection <a name="color"></a>

[Back to content table](#content)

* `cv2.namedWindow(name, flags)`  
    * name: Window's name.  
    * flags: Flags of the window. See the documentation for more detail.
      
        
* `cv2.resizeWindow(name, width, height)`  
    * name: The name of the window to be resized. 
    * width: Width after resize.  
    * height: Height after resize.  
      
        
* `cv2.createTracker(trackbar_name, window_name, min_value, max_value, callback)`  
    * trackbar_name: The name of new trackbar.  
    * window_name: The window to add trackbar.  
    * min_value: Minimum value of the trackbar. 
    * max_value: Maximum value of the trackbar.  
    * callback: Function to be called when the value of trackbar has changed.  
      
        
* `cv2.getTrackbarPos(trackbar_name, window_name)`
    Get the result of the trackbar
    * trackbar_name: The target trackbar's name.  
    * window_name: The window where the target trackbar exists.  
    
      
* `cv2.inRange(img, lower_value, upper_value)`  
    * img: Image to filter.  
    * Change the value between lower_value and upper_value to 255 and others to 0.  
    
      
* `cv2.bitwise_and(img1, img2, mask)`
    * img1: First input image.  
    * img2: Second input image.  
    * mask: After doing bitwise_and of img1 and img2 get temp1, do bitwise and with temp1 and get result.

In [ ]:
import cv2
import numpy as np


# Callback function
def empty(x):
    pass


# Create window
cv2.namedWindow("TrackBars", cv2.WINDOW_NORMAL)
cv2.resizeWindow("TrackBars", 1000, 40)

# Create trackbar
cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, empty)
cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, empty)
cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, empty)
cv2.createTrackbar("Value Min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("Value Max", "TrackBars", 255, 255, empty)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import cv2
import numpy as np

# Convert image to HSV format
img = cv2.imread("resources/car.jpg")
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Using the mask to filter the color
lower = np.array([0, 93, 0])
upper = np.array([10, 255, 255])
mask = cv2.inRange(img_hsv, lower, upper)

# Result image
img_result = cv2.bitwise_and(img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR))

result = np.hstack([img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), img_result])
cv2.imshow("Mask image", result)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

cv2.imwrite("result/bitwise.jpg", result)

In [ ]:
import cv2
import numpy as np


def empty(x):
    pass


# Create window
cv2.namedWindow("TrackBars", cv2.WINDOW_NORMAL)
cv2.resizeWindow("TrackBars", 1000, 100)

# Create trackbar
cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, empty)
cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, empty)
cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, empty)
cv2.createTrackbar("Value Min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("Value Max", "TrackBars", 255, 255, empty)

# Convert image to HSV format
img = cv2.imread("resources/car.jpg")
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
while True:
    # Get the trackbar value
    h_lower = cv2.getTrackbarPos("Hue Min", "TrackBars")
    h_upper = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_lower = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_upper = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_lower = cv2.getTrackbarPos("Value Min", "TrackBars")
    v_upper = cv2.getTrackbarPos("Value Max", "TrackBars")
    
    # Print out the value
    # print(h_min, h_max, s_min, s_max, v_min, v_max)
    
    # Using the mask to filter the color
    lower = np.array([h_lower, s_lower, v_lower])
    upper = np.array([h_upper, s_upper, v_upper])
    mask = cv2.inRange(img_hsv, lower, upper)
    
    # Result image
    img_result = cv2.bitwise_and(img, img, mask=mask)
    
    # Show the image
    img_stack = stackImages(0.5, [[img, img_hsv], [mask, img_result]])
    cv2.imshow("Image", img_stack)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break;
        

print(h_lower, s_lower, v_lower)
print(h_upper, s_upper, v_upper)

cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!

## Contour / Shape dectection <a name="contour"></a>

[Back to content table](#content)
  
* `cv2.findContours(img, mode, method)` [Documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#findcontours)  
    * img: Image to find contour.  
    * mode: Contour retrieval mode. See the detail in documentation.  
    * Method: Contour approximation method. See the detail in documentation.  
    
  
* `cv2.contourArea(contour)` [Documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#contourarea)  
    * contour: Contour get from cv2.findContours(). In fact, it could be 2D array which shape is (n, 2) for storing the vertex information (point vector).
    
      
* `cv2.drawContours(img_output, contour, contourIdx, color, thickness)` [Documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#drawcontours)  
    * img_output: Draw on which image.  
    * contour: Point vectors (Same as above).  
    * contourIdx: Index for which contour to draw. If it is negative, draw all the contours.  
    
      
* `cv2.arcLength(contour, closed)` [Documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#arclength)  
    * closed: Flag indicating whether the curve is closed or not.  
    
      
* `cv2.approxPolyDP(contour, epsilon, closed)` [Documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#approxpolydp)  
    * epsilon: Specifying the approximation accuracy.  
    
      
* `cv2.boundingRect(points)` [Documentation](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#boundingrect)  
    * points: Point vectors.

In [ ]:
def get_contours(img):
    # Retreive the outer contours
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours:
        # Get the area of the closed contour
        area = cv2.contourArea(cnt)
        
        # Clean noise by setting the threshold of area
        if area < 800:
            continue
         
        # Draw the contour
        cv2.drawContours(img_contour, cnt, -1, (255, 0, 0), 3)
        
        # Connt the peripheral 
        peri = cv2.arcLength(cnt, True)
        
        # Get the vertex of the shape
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
        
        # The number of the sides
        obj_cor = len(approx)
        
        # Get the bounding box of the shape
        x, y, w, h = cv2.boundingRect(approx)
        
        # Detect the shape
        if obj_cor == 3:
            object_type = "Tri"
        elif obj_cor == 4:
            # Check whether is circle or rectangle
            asp_ratio = w / h
            
            if (asp_ratio > 0.95) & (asp_ratio < 1.05):
                object_type = "Square"
            else:
                object_type = "Rectangle"
        elif obj_cor == 5:
            object_type = "Pentagon"
        elif obj_cor == 6:
            object_type = "Hexagon"
        else:
            object_type = "Circle"
        
        # Drow the bounding box and put the shape text on it
        cv2.rectangle(img_contour, (x, y), (x + w, y + h), (0, 0, 255), 3)
        cv2.putText(img_contour, object_type,
                    (x + (w // 2) - 20, y - 15), cv2.FONT_HERSHEY_COMPLEX, 0.5,
                    (0, 0, 0), 2)

In [ ]:
import cv2
import numpy as np

img = cv2.imread("resources/shapes.jpg")

# Change the image to gray scale and blur the image
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.GaussianBlur(img_gray, (3, 3), 1)

# Edge detection
img_canny = cv2.Canny(img_blur, 50, 50)

# Copy the original image
img_contour = img.copy()

# Get contours
get_contours(img_canny)

# Show the image
img_stack = stackImages(0.5, [[img, img_blur], [img_canny, img_contour]])
cv2.imshow("Result", img_stack)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

cv2.imwrite("contours.jpg", img_stack)

In [ ]:
# Do what you want to test!

## Object detection <a name="object"></a>

[Back to content table](#content)

* `cv2.CascadeClassifier(file)` [Documentation](https://docs.opencv.org/2.4/modules/objdetect/doc/cascade_classification.html#cascadeclassifier-cascadeclassifier)   
    Loads a classifier from a file.
    * file: Place to load the xml file.  
    
      
* `cv2.CascadeClassifier.detectMultiScale(img, scaleFactor, minNeighbors)` [Documentation](https://docs.opencv.org/2.4/modules/objdetect/doc/cascade_classification.html#cascadeclassifier-detectmultiscale)  
    * img: Image to detect.
    * scaleFactor: Specifying how much the image size is reduced at each image scale.  
    * minNeighbors: Specifying how many neighbors each candidate rectangle should have to retain it.

In [ ]:
import cv2
import numpy as np

# Read in the xml file
face_cascade = cv2.CascadeClassifier("cascade_files/haarcascade_frontalface_default.xml")

img = cv2.imread("resources/me.jpg")

# Convert into gray scale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Dectect the face
faces = face_cascade.detectMultiScale(img_gray, 1.1, 5)

# Draw the bounding boxes
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
cv2.imshow("Result", img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Do what you want to test!